In [1]:
import pandas as pd
import numpy as np

from typing import Tuple, Dict, List

In [2]:
INPUT_FILE = "inventory_raw.csv"

In [3]:
raw_data = pd.read_csv(INPUT_FILE)

In [4]:
raw_data

,source_row_id,ip,hostname,fqdn,mac,owner,device_type,site,notes
0,1,192.168.010.005,HOST01,NaN,AA-BB-CC-DD-EE-FF,priya (platform) priya@corp.example.com,server,BLR Campus,db host
1,2,10.0.1.300,host-02,host-02.local,11-22-33-44-55-66,ops,NaN,HQ Bldg 1,edge gw?
2,3,10.0.1,host03,NaN,aabb.ccdd.eeff,jane@corp.example.com,switch,HQ-BUILDING-1,NaN
3,4,10.0.1.1.2,printer-01,NaN,00:11:22:33:44:55,Facilities,printer,HQ,NaN
4,5,fe80::1%eth0,iot-cam01,NaN,00:aa:bb:cc:dd:ee,sec,iot,Lab-1,camera PoE on port 3
5,6,127.0.0.1,local-test,NaN,NaN,NaN,NaN,NaN,NaN
6,7,169.254.10.20,host-apipa,NaN,NaN,NaN,NaN,NaN,NaN
7,8,10.10.10.10,srv-10,NaN,NaN,platform,server,BLR campus,NaN
8,9,abc.def.ghi.jkl,badhost,NaN,NaN,NaN,NaN,NaN,NaN
9,10,192.168.1.-1,neg,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(list(raw_data.columns))

['source_row_id', 'ip', 'hostname', 'fqdn', 'mac', 'owner', 'device_type', 'site', 'notes']


In [6]:
# Set index to source_row_id
raw_data = raw_data.set_index("source_row_id")

In [7]:
raw_data

,ip,hostname,fqdn,mac,owner,device_type,site,notes
source_row_id,,,,,,,,
1,192.168.010.005,HOST01,NaN,AA-BB-CC-DD-EE-FF,priya (platform) priya@corp.example.com,server,BLR Campus,db host
2,10.0.1.300,host-02,host-02.local,11-22-33-44-55-66,ops,NaN,HQ Bldg 1,edge gw?
3,10.0.1,host03,NaN,aabb.ccdd.eeff,jane@corp.example.com,switch,HQ-BUILDING-1,NaN
4,10.0.1.1.2,printer-01,NaN,00:11:22:33:44:55,Facilities,printer,HQ,NaN
5,fe80::1%eth0,iot-cam01,NaN,00:aa:bb:cc:dd:ee,sec,iot,Lab-1,camera PoE on port 3
6,127.0.0.1,local-test,NaN,NaN,NaN,NaN,NaN,NaN
7,169.254.10.20,host-apipa,NaN,NaN,NaN,NaN,NaN,NaN
8,10.10.10.10,srv-10,NaN,NaN,platform,server,BLR campus,NaN
9,abc.def.ghi.jkl,badhost,NaN,NaN,NaN,NaN,NaN,NaN


# IP Validation

In [8]:
def trim_ip_str(ip: str) -> str:
    try:
        return ip.strip()
    except AttributeError:
        return ""

In [9]:
def validate_and_label_ipv4(ip: str) -> Tuple[str, str]:
    if ip == "":
        return ip, "empty_string"
    if ":" in ip:
        return ip, "ipv6_or_mixed_non_ipv4"
    if "." not in ip:
        return ip, "no_octet_separation"
    parts = ip.split(".")
    if len(parts) != 4:
        return ip, "wrong_part_count"
    canonical_parts = []
    for part in parts:
        part = part.strip()
        if part == "":
            return ip, "empty_octet"
        if not (part.lstrip("+").isdigit() and not part.startswith("-")):
            return ip, "non_numeric_or_negative"
        try:
            v = int(part, 10)
        except ValueError:
            return ip, "non_decimal_format"
        if v < 0 or v > 255:
            return ip, "octet_out_of_range"
        canonical_parts.append(str(v))
    return '.'.join(canonical_parts), "ok"

In [10]:
def determine_reverse_ptr_ipv4(ip: str, validation_label: str) -> str:
    if validation_label != "ok":
        return ""
    parts = ip.split(".")
    reversed_parts = parts[::-1]
    return ".".join(reversed_parts) + ".in-addr.arpa"

In [11]:
def classify_ipv4(ip: str, validation_label: str) -> str:
    if validation_label != "ok":
        return "unclassified"
    a, b, c, d = list(map(int, ip.split(".")))
    classification = ""
    if ip == "0.0.0.0" :
        classification = "unspecified"
    elif ip == "255.255.255.255":
        classification = "limited_broadcast"
    elif a == 127:
        classification = "loopback"
    elif a == 169 and b == 254:
        classification = "link_local_apipa"
    elif 224 <= a <= 239:
        classification = "multicast"
    elif 240 <= a <= 255 and ip != "255.255.255.255":
        classification = "reserved"
    elif a == 10 or (a == 172 and 16 <= b <= 31) or (a == 192 and b == 168):
        classification = "private"
    else:
        classification = "public_or_other"
    return classification

In [12]:
def determine_subnet(ip: str, classification: str) -> str:
    if classification == "unclassified":
        return ""
    if classification in ["limited_broadcast", "unspecified", "unclassified", "multicast", "reserved"]:
        return ""
    elif classification == "loopback":
        mask = "8"
        subnet_ip = f"{'.'.join(ip.split(".")[:3])}.0/{mask}"
        return subnet_ip
    elif classification == "private":
        mask = "24"
        subnet_ip = f"{ip}/{mask}"
        return subnet_ip
    elif classification == "link_local_apipa":
        mask = "16"
        subnet_ip = f"{'.'.join(ip.split(".")[:2])}.0.0/{mask}"
        return subnet_ip
    else:
        return ""

In [13]:
def process_ipv4(ip: str) -> Dict:
    steps = []
    notes = []
    trimmed_ip = trim_ip_str(ip)
    steps.append("ip_trim")
    trimmed_ip, validation_label = validate_and_label_ipv4(trimmed_ip)
    if validation_label == "ok":
        steps.append("ip_parse")
        steps.append("ip_normalize")
        reverse_ptr = determine_reverse_ptr_ipv4(trimmed_ip, validation_label)
        steps.append("ip_reverse_ptr_determine")
        classification = classify_ipv4(trimmed_ip, validation_label)
        steps.append("ip_classify")
        subnet = determine_subnet(trimmed_ip, classification)
        steps.append("ip_subnet_determine")
        ip_out = trimmed_ip
        ip_valid = "True"
        ip_version = "4"
        ip_reverse_ptr = reverse_ptr
        ip_classification = classification
        subnet_cidr = subnet
        ip_issues = None
        ip_recommended_action = None
    else:
        ip_out = str(ip).strip()
        ip_classification = ""
        ip_valid = "False"
        ip_version = ""
        ip_reverse_ptr = ""
        ip_classification = ""
        subnet_cidr = ""
        ip_issues = validation_label
        ip_recommended_action = "Correct IP or mark record for revision"
        steps.append(f"ip_invalid_{validation_label}")
    return {
        "ip_out": ip_out,
        "ip_valid": ip_valid,
        "ip_version": ip_version,
        "ip_reverse_ptr": ip_reverse_ptr,
        "ip_classification": ip_classification,
        "subnet_cidr": subnet_cidr,
        "ip_issues": ip_issues,
        "ip_recommended_action": ip_recommended_action,
        "ip_normalization_steps": "|".join(steps),
        # Add row ID
    }

In [14]:
# Test
ip1 = "192.168.1.1"
ip2 = " 192.168.1.1"
ip3 = " 192. 168.1.1"
ip4 = "192. 168"
ip5 = "0.0.0.0"
ip6 = "130.203.12.23"

# trimmed_ip = trim_ip_str(ip3)
# print(f"Trimmed IP: {trimmed_ip}")
# ip, validation_label = validate_and_label_ipv4(trimmed_ip)
# print(f"Validated IP: {ip}, {validation_label}")
# classification = classify_ipv4(ip, validation_label)
# print(f"Classification of IP: {classification}")
# subnet = determine_subnet(ip, classification)
# print(f"IP with subnet mask: {subnet}")

print(process_ipv4(ip4))

{'ip_out': '192. 168', 'ip_valid': 'False', 'ip_version': '', 'ip_reverse_ptr': '', 'ip_classification': '', 'subnet_cidr': '', 'ip_issues': 'wrong_part_count', 'ip_recommended_action': 'Correct IP or mark record for revision', 'ip_normalization_steps': 'ip_trim|ip_invalid_wrong_part_count'}


# MAC Validation

In [15]:
def trim_mac_str(mac: str) -> str:
    try:
        return str(mac).strip()
    except Exception:
        return ""

In [16]:
def is_valid_hex(s: str) -> bool:
    HEX = set("01223456789abcdefABCDEF")
    return s != "" and all(c in HEX for c in s)

ChatGPT prompt:

The functions above I want formatted something like this: def trim_ip_str(ip: str) -> str: try: return ip.strip() except AttributeError: return "" def validate_and_label_ipv4(ip: str) -> Tuple[str, str]: if ip == "": return ip, "empty_string" if ":" in ip: return ip, "ipv6_or_mixed_non_ipv4" if "." not in ip: return ip, "no_octet_separation" parts = ip.split(".") if len(parts) != 4: return ip, "wrong_part_count" canonical_parts = [] for part in parts: part = part.strip() if part == "": return ip, "empty_octet" if not (part.lstrip("+").isdigit() and not part.startswith("-")): return ip, "non_numeric_or_negative" try: v = int(part, 10) except ValueError: return ip, "non_decimal_format" if v < 0 or v > 255: return ip, "octet_out_of_range" canonical_parts.append(str(v)) return '.'.join(canonical_parts), "ok" def classify_ipv4(ip: str, validation_label: str) -> str: if validation_label != "ok": return "unclassified" a, b, c, d = list(map(int, ip.split("."))) classification = "" if ip == "0.0.0.0" : classification = "unspecified" elif ip == "255.255.255.255": classification = "limited_broadcast" elif a == 127: classification = "loopback" elif a == 169 and b == 254: classification = "link_local_apipa" elif 224 <= a <= 239: classification = "multicast" elif 240 <= a <= 255 and ip != "255.255.255.255": classification = "reserved" elif a == 10 or (a == 172 and 16 <= b <= 31) or (a == 192 and b == 168): classification = "private" else: classification = "public_or_other" return classification The trim function should just trim the string, the validate function should check only for validity and return the validated and normalized MAC address and a label for any error encountered, the classify function should only return classifications like "eui64" or "eui48" The function to check valid hex string is: def is_valid_hex(s: str) -> bool: HEX = set("01223456789abcdefABCDEF") return s != "" and all(c in HEX for c in s)


In [17]:
def validate_and_label_mac(mac: str) -> Tuple[str, str]:
    """
    Returns (canonical_mac, label).
      - On success: ('aa:bb:cc:dd:ee:ff', 'ok') or EUI-64 equivalent.
      - On failure: (original_input, '<error_label>').
    Canonical form: lowercase, colon-separated. Accepts:
      - aa:bb:cc:dd:ee:ff  | aa-bb-cc-dd-ee-ff
      - aabb.ccdd.eeff     | aabb.ccdd.eeff.gghh
      - aabbccddeeff       | aabbccddeeffgghh
    """
    if mac is None:
        return "", "missing"
    s = trim_mac_str(mac)
    if s == "":
        return s, "empty_string"

    has_colon = ":" in s
    has_dash  = "-" in s
    has_dot   = "." in s

    # reject mixed separators
    if sum([has_colon, has_dash, has_dot]) > 1:
        return s, "mixed_separators"

    octets: List[str] = []

    if has_colon or has_dash:
        sep = ":" if has_colon else "-"
        parts = s.split(sep)
        if len(parts) not in (6, 8):
            return s, "wrong_group_count"
        for p in parts:
            p = p.strip()
            if len(p) != 2 or not is_valid_hex(p):
                return s, "bad_octet_hex"
            octets.append(p.lower())

    elif has_dot:
        # Cisco-style: aabb.ccdd.eeff(.gghh)
        parts = s.split(".")
        if len(parts) not in (3, 4):
            return s, "wrong_group_count_dot"
        for grp in parts:
            grp = grp.strip()
            if len(grp) != 4 or not is_valid_hex(grp):
                return s, "bad_group_hex_dot"
            octets.extend([grp[0:2].lower(), grp[2:4].lower()])

    else:
        # No separators: 12 (EUI-48) or 16 (EUI-64) hex chars
        if not is_valid_hex(s):
            return s, "non_hex_chars"
        if len(s) not in (12, 16):
            return s, "wrong_length_no_separators"
        octets = [s[i:i+2].lower() for i in range(0, len(s), 2)]

    if len(octets) not in (6, 8):
        return s, "not_6_or_8_octets"

    canonical = ":".join(octets)
    return canonical, "ok"

def classify_mac(mac: str, validation_label: str) -> str:
    """
    Only returns 'eui48' or 'eui64' (or 'unclassified' if not ok).
    """
    if validation_label != "ok":
        return "unclassified"
    count = len(mac.split(":"))
    if count == 6:
        return "eui48"
    if count == 8:
        return "eui64"
    return "unclassified"

In [18]:
def process_mac(mac: str) -> Dict:
    steps = []
    notes = []
    trimmed_mac = trim_mac_str(mac)
    steps.append("mac_trim")
    trimmed_mac, validation_label = validate_and_label_mac(trimmed_mac)
    if validation_label == "ok":
        steps.append("mac_parse")
        steps.append("mac_normalize")
        classification = classify_mac(trimmed_mac, validation_label)
        steps.append("mac_classify")
        mac_out = trimmed_mac
        mac_valid = "True"
        mac_kind = classification
        mac_issues = None
        mac_recommended_action = None
    else:
        mac_out = str(mac).strip()
        mac_valid = "False"
        mac_kind = ""
        mac_issues = validation_label
        mac_recommended_action = "Correct MAC or mark record for revision"
        steps.append(f"mac_invalid_{validation_label}")
    return {
        "mac_out": mac_out,
        "mac_valid": mac_valid,
        "mac_kind": mac_kind,
        "mac_issues": mac_issues,
        "mac_recommended_action": mac_recommended_action,
        "mac_normalization_steps": "|".join(steps),
        # Add row ID
    }

In [19]:
# # Test
# for mac in raw_data["mac"]:
#     print(f"Working on {mac}")
#     print(process_mac(mac))

# GPT Client

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [21]:
class GPTClient:
    def __init__(self, model="gpt-4o-mini", temperature=0.2):
        # Load environment variables from .env
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY not found in .env")

        # Initialize the OpenAI client
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.temperature = temperature

    def generate(self, system_prompt: str, prompt: str) -> str:
        """Send a prompt and return the model's text output."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=self.temperature,
        )
        response = response.choices[0].message.content.strip()
        if response.startswith("```"):
            response = response.strip("`").replace("json", "", 1).strip()

        return json.loads(response)


In [22]:
gpt = GPTClient(model="gpt-4o-mini", temperature=0.2)

# Owner parsing

In [23]:
system_prompt = '''
You specialize in network analytics
'''
owner_prompt = '''
Given the following string, I want you to parse it to extract:
- An email address
- A name
- A team name

I want you to return the response to me in a JSON format containing:
- owner_out (Capitalize owner name if possible, may be possible to obtain from email address too)
- owner_email
- owner_team

I want only the JSON and nothing else

Wherever impossible to do so, return empty strings within the JSON fields

String:
'''

In [24]:
def trim_owner_str(owner: str) -> str:
    try:
        return str(owner).strip()
    except Exception:
        return ""

In [52]:
def process_owner(llm: GPTClient, owner: str, owner_prompt: str, system_prompt: str) -> Dict:
    steps = []
    notes = []
    trimmed_owner = trim_owner_str(owner)
    steps.append("owner_trim")
    owner_prompt_augmented = owner_prompt + trimmed_owner
    owner = llm.generate(system_prompt, owner_prompt_augmented)
    steps.append("owner_parse")
    if any(v == "" for v in owner.values()):
        owner_issues = "Missing owner fields"
        owner_recommended_action = "Correct owner or mark record for revision"
        steps.append(f"owner_invalid_missing_owner_fields")
    else:
        owner_issues = None
        owner_recommended_action = None
    return {
        **owner,
        "owner_issues": owner_issues,
        "owner_recommended_action": owner_recommended_action,
        "owner_normalization_steps": "|".join(steps)
    }

In [54]:
# Test
for owner in raw_data["owner"]:
    print(f"Working on {owner}")
    print(process_owner(gpt, owner, owner_prompt, system_prompt))

Working on priya (platform) priya@corp.example.com
{'owner_out': 'Priya', 'owner_email': 'priya@corp.example.com', 'owner_team': 'platform', 'owner_issues': None, 'owner_recommended_action': None, 'owner_normalization_steps': 'owner_trim|owner_parse'}
Working on ops
{'owner_out': '', 'owner_email': '', 'owner_team': '', 'owner_issues': 'Missing owner fields', 'owner_recommended_action': 'Correct owner or mark record for revision', 'owner_normalization_steps': 'owner_trim|owner_parse|owner_invalid_missing_owner_fields'}
Working on jane@corp.example.com
{'owner_out': 'Jane', 'owner_email': 'jane@corp.example.com', 'owner_team': '', 'owner_issues': 'Missing owner fields', 'owner_recommended_action': 'Correct owner or mark record for revision', 'owner_normalization_steps': 'owner_trim|owner_parse|owner_invalid_missing_owner_fields'}
Working on Facilities
{'owner_out': '', 'owner_email': '', 'owner_team': 'Facilities', 'owner_issues': 'Missing owner fields', 'owner_recommended_action': 'Cor

KeyboardInterrupt: 

# Device type parsing

In [55]:
def trim_device_type_str(device_type: str) -> str:
    try:
        return str(device_type).strip()
    except Exception:
        return ""

In [56]:
def process_device(llm: GPTClient, device: str, hostname: str, notes: str, device_prompt: str, system_prompt: str) -> Dict:
    steps = []
    notes = []
    steps.append("device_trim")
    device_prompt_augmented = device_prompt + f"Hostname: {hostname} Device Type: {trim_device_type_str(device)} Notes: {notes}"
    device = llm.generate(system_prompt, device_prompt_augmented)
    steps.append("device_parse")
    if any(v == "" for v in device.values()):
        device_issues = "Missing device fields"
        device_recommended_action = "Correct device or mark record for revision"
        steps.append(f"device_invalid_missing_device_fields")
    else:
        device_issues = None
        device_recommended_action = None
    return {
        **device,
        "device_issues": device_issues,
        "device_recommended_action": device_recommended_action,
        "device_normalization_steps": "|".join(steps)
    }

In [57]:
system_prompt = '''
You specialize in network analytics
'''
device_prompt = '''
Given the following string, I want you to parse it to extract:
- Device Type (based on Hostname and Device Type and Notes)
- Confidence score (low, high, mid) based on your classification, be very critical of this

I want you to return the response to me in a JSON format containing:
- device_out
- device_type_confidence

I want only the JSON and nothing else

Wherever impossible to do so, return empty strings within the JSON fields

String:
'''

In [58]:
# Test
for hostname, device, notes in zip(raw_data["hostname"], raw_data["device_type"], raw_data["notes"]):
    print(f"Working on {hostname} and {device}")
    print(process_device(gpt, device, hostname, notes, device_prompt, system_prompt))

Working on HOST01 and server
{'device_out': 'server', 'device_type_confidence': 'high', 'device_issues': None, 'device_recommended_action': None, 'device_normalization_steps': 'device_trim|device_parse'}
Working on host-02 and nan
{'device_out': '', 'device_type_confidence': '', 'device_issues': 'Missing device fields', 'device_recommended_action': 'Correct device or mark record for revision', 'device_normalization_steps': 'device_trim|device_parse|device_invalid_missing_device_fields'}
Working on host03 and switch


KeyboardInterrupt: 

# Site normalization

In [31]:
import re

In [32]:
def normalize_site_name(name: str) -> str:
    steps = []
    if not name or not isinstance(name, str):
        return {
            "site_out": "",
            "site_normalization_steps": "site_invalid_missing_site"
        }

    # Mapping of common abbreviations to full forms
    replacements = {
        r"\bBldg\b": "Building",
        r"\bBLR\b": "Bangalore",
        r"\bDC\b": "Datacenter",
        r"\bHQ\b": "Headquarters",
        r"\bLab\b": "Laboratory",
        r"\bCampus\b": "Campus",  # keep capitalization consistent
    }

    s = name.strip()

    # Apply replacements (case-insensitive)
    for pattern, full in replacements.items():
        s = re.sub(pattern, full, s, flags=re.IGNORECASE)

    steps.append("site_replace_common_abbreviations")
    # Replace spaces/underscores with hyphens
    s = re.sub(r"[ _]+", "-", s)
    steps.append("site_replace_common_abbreviations")
    
    # Remove duplicate hyphens
    s = re.sub(r"-{2,}", "-", s)
    steps.append("site_replace_whitespace_with_hypen")

    # Normalize capitalization (title case or upper depending on your style)
    s = s.title()
    steps.append("site_capitalize")

    if s == "":
        site_issues = "Missing site fields"
        site_recommended_action = "Correct site or mark record for revision"
        steps.append("site_invalid_missing_site_fields")
    else:
        site_issues = None
        site_recommended_action = None

    return {
        "site_out": s,
        "site_issues": site_issues,
        "site_recommended_action": site_recommended_action,
        "site_normalization_steps": "|".join(steps)
    }

In [33]:
# for site in raw_data["site"]:
#     print(f"Working on {site}")
#     print(normalize_site_name(site))

# Hostname and FQDN

ChatGPT prompt:

Write Python functions to trim hostname and FQDN strings, validate them and classify them based on the funciton prototypes above

In [34]:
# ---------- TRIM ----------
def trim_dns_str(s: str) -> str:
    try:
        return s.strip()
    except AttributeError:
        return ""

# ---------- CORE HELPERS ----------
def _label_to_idna_ascii(label: str) -> Tuple[str, str]:
    """
    Convert a (possibly unicode) label to IDNA ASCII (xn--...).
    Returns (ascii_label, err_label). err_label == "" means OK.
    """
    try:
        ascii_label = label.encode("idna").decode("ascii")
        return ascii_label, ""
    except Exception:
        return label, "idna_encode_failed"

def _is_valid_label_ascii(label: str, allow_underscore: bool) -> str:
    """
    Validate an ASCII (already IDNA-encoded) DNS label per RFC 1123:
      - 1..63 chars
      - letters/digits/hyphen, optional underscore if allowed
      - cannot start or end with '-'
    Return "" if OK, else an error label.
    """
    if label == "":
        return "empty_label"
    if len(label) > 63:
        return "label_too_long"
    if label[0] == "-" or label[-1] == "-":
        return "label_starts_or_ends_with_hyphen"
    for ch in label:
        if ch.isalnum() or ch == "-" or (allow_underscore and ch == "_"):
            continue
        return "invalid_characters"
    return ""

# ---------- HOSTNAME (SINGLE LABEL) ----------
def validate_and_label_hostname_label(label: str, *, allow_underscore: bool = False) -> Tuple[str, str]:
    """
    Validate a single hostname label.
    Returns (normalized_label, 'ok') on success, or (original_input, '<error>') on failure.
    Normalization: IDNA → ASCII, then lowercase.
    """
    s = trim_dns_str(label)
    if s == "":
        return s, "empty_string"
    ascii_label, err = _label_to_idna_ascii(s)
    if err:
        return s, err
    ascii_label = ascii_label.lower()
    e = _is_valid_label_ascii(ascii_label, allow_underscore)
    if e:
        return s, e
    return ascii_label, "ok"

def classify_hostname_label(normalized_label: str, validation_label: str) -> str:
    """
    Return 'single_label' when validation_label == 'ok', else 'unclassified'.
    """
    return "single_label" if validation_label == "ok" else "unclassified"

# ---------- FQDN (ONE OR MORE LABELS, OPTIONAL TRAILING DOT) ----------
def validate_and_label_fqdn(
    name: str,
    *,
    allow_underscore: bool = False,
    require_at_least_two_labels: bool = False,
    forbid_numeric_tld: bool = False
) -> Tuple[str, str]:
    """
    Validate a hostname/FQDN (e.g., 'api.example.com' or 'api.example.com.').
    Returns (normalized_ascii, 'ok' | 'ok_absolute') on success, or (original_input, '<error>').

    Normalization:
      - Each label IDNA-encoded to ASCII and lowercased.
      - Trailing dot (absolute FQDN) is removed from the returned value; 'ok_absolute' signals it was present.

    Constraints:
      - Per-label rules per RFC 1123 (with optional underscores).
      - Total length (joined with dots, NO trailing dot) ≤ 253.
      - If require_at_least_two_labels=True, single-label names are rejected.
      - If forbid_numeric_tld=True, the last label cannot be all digits.
    """
    s = trim_dns_str(name)
    if s == "":
        return s, "empty_string"

    absolute = s.endswith(".")
    if absolute:
        s = s[:-1]  # strip for validation/length checks

    raw_labels = s.split(".")
    if any(lbl == "" for lbl in raw_labels):
        return name, "empty_label_in_sequence"

    ascii_labels: List[str] = []
    for lbl in raw_labels:
        a, err = _label_to_idna_ascii(lbl)
        if err:
            return name, err
        a = a.lower()
        e = _is_valid_label_ascii(a, allow_underscore)
        if e:
            return name, e
        ascii_labels.append(a)

    if require_at_least_two_labels and len(ascii_labels) < 2:
        return name, "requires_at_least_two_labels"

    if forbid_numeric_tld and ascii_labels and ascii_labels[-1].isdigit():
        return name, "numeric_tld_forbidden"

    joined = ".".join(ascii_labels)
    if len(joined) > 253:
        return name, "fqdn_too_long"

    return joined, ("ok_absolute" if absolute else "ok")

def classify_fqdn(normalized_name: str, validation_label: str) -> str:
    """
    Return:
      - 'absolute_fqdn' if validation_label == 'ok_absolute'
      - 'fqdn' if validation_label == 'ok' and there's at least one dot
      - 'single_label' if validation_label == 'ok' and no dots
      - 'unclassified' otherwise
    """
    if validation_label == "ok_absolute":
        return "absolute_fqdn"
    if validation_label == "ok":
        return "fqdn" if "." in normalized_name else "single_label"
    return "unclassified"

# ---------- OPTIONAL: AUTO-ROUTER (if your column may be either) ----------
def validate_and_label_dns_name(
    name: str,
    *,
    allow_underscore: bool = False,
    require_at_least_two_labels_for_fqdn: bool = False,
    forbid_numeric_tld: bool = False
) -> Tuple[str, str, str]:
    """
    Convenience: tries FQDN validation first (accepts single labels), returns
    (normalized_value, validation_label, kind)
      - kind in {'absolute_fqdn','fqdn','single_label','unclassified'}
    """
    normalized, label = validate_and_label_fqdn(
        name,
        allow_underscore=allow_underscore,
        require_at_least_two_labels=require_at_least_two_labels_for_fqdn,
        forbid_numeric_tld=forbid_numeric_tld,
    )
    kind = classify_fqdn(normalized, label)
    return normalized, label, kind

In [35]:
def process_hostname(hostname: str) -> Dict:
    steps = []
    notes = []
    hostname_normalized, hostname_label = validate_and_label_hostname_label(hostname)
    steps.append("hostname_normalize")
    steps.append("hostname_label")
    hostname_classification = classify_hostname_label(hostname_normalized, hostname_label)
    steps.append("hostname_classify")
    if hostname_label == "ok":
        hostname_out = hostname_normalized
        hostname_valid = "True"
        hostname_kind = hostname_classification
        hostname_issues = None
        hostname_recommended_action = None
    else:
        hostname_out = hostname
        hostname_valid = "False"
        hostname_kind = ""
        hostname_issues = hostname_label
        hostname_recommended_action = "Correct hostname or mark record for revision"
        steps.append(f"hostname_invalid_{hostname_issues}")
    return {
        "hostname_out": hostname_out,
        "hostname_valid": hostname_valid,
        "hostname_kind": hostname_kind,
        "hostname_issues": hostname_issues,
        "hostname_recommended_action": hostname_recommended_action,
        "hostname_normalization_steps": "|".join(steps)
    }

In [36]:
def process_fqdn(fqdn: str) -> Dict:
    steps = []
    notes = []
    fqdn_normalized, fqdn_label = validate_and_label_fqdn(fqdn)
    steps.append("fqdn_normalize")
    steps.append("fqdn_label")
    fqdn_classification = classify_fqdn(fqdn_normalized, fqdn_label)
    steps.append("fqdn_classify")
    if fqdn_label == "ok":
        fqdn_out = fqdn_normalized
        fqdn_valid = "True"
        fqdn_kind = fqdn_classification
        fqdn_issues = None
        fqdn_recommended_action = None
    else:
        fqdn_out = fqdn
        fqdn_valid = "False"
        fqdn_kind = ""
        fqdn_issues = fqdn_label
        fqdn_recommended_action = "Correct FQDN or mark record for revision"
        steps.append(f"fqdn_invalid_{fqdn_issues}")
    return {
        "fqdn_out": fqdn_out,
        "fqdn_valid": fqdn_valid,
        "fqdn_kind": fqdn_kind,
        "fqdn_issues": fqdn_issues,
        "fqdn_recommended_action": fqdn_recommended_action,
        "fqdn_normalization_steps": "|".join(steps)
    }

In [37]:
# # Test
# for hostname, fqdn in zip(raw_data["hostname"], raw_data["fqdn"]):
#     print(f"Working on {hostname} and {fqdn}")
#     # hostname_normalized, hostname_label = validate_and_label_hostname_label(hostname)
#     # hostname_classification = classify_hostname_label(hostname_normalized, hostname_label)
#     # print(f"Hostname normalized: {hostname_normalized}, hostname label: {hostname_label}, hostname classification: {hostname_classification}")
#     print(process_hostname(hostname))
#     # fqdn_normalized, fqdn_label = validate_and_label_fqdn(fqdn)
#     # fqdn_classification = classify_fqdn(fqdn_normalized, fqdn_label)
#     # print(f"FQDN normalized: {fqdn_normalized}, FQDN label: {fqdn_label}, FQDN classification: {fqdn_classification}")
#     print(process_fqdn(fqdn))

# Process each row

In [38]:
def apply_and_expand(df: pd.DataFrame, func, input_cols: list[str], **kwargs) -> pd.DataFrame:
    """
    Apply a function that takes multiple columns from each row and returns a dict.
    The returned dict's keys become new columns in the DataFrame.

    Args:
        df: Input DataFrame
        func: Function to apply (should accept *args or a row)
        input_cols: List of column names to pass to func

    Returns:
        DataFrame with new columns added
    """
    # Apply the function row-wise, passing in the specified columns
    result_df = (
        df[input_cols]
        .apply(lambda row: func(*row, **kwargs), axis=1)
        .apply(pd.Series)
    )

    # Join results back to the original DataFrame
    return df.join(result_df)

In [39]:
ip_norm_df = apply_and_expand(raw_data, process_ipv4, input_cols=["ip"])
mac_norm_df = apply_and_expand(ip_norm_df, process_mac, input_cols=["mac"])
site_norm_df = apply_and_expand(mac_norm_df, normalize_site_name, input_cols=["site"])
hostname_norm_df = apply_and_expand(site_norm_df, process_hostname, input_cols=["hostname"])
fqdn_norm_df = apply_and_expand(hostname_norm_df, process_fqdn, input_cols=["fqdn"])
owner_norm_df = apply_and_expand(fqdn_norm_df, process_owner, input_cols=["owner"], system_prompt=system_prompt, owner_prompt=owner_prompt)
device_norm_df = apply_and_expand(owner_norm_df, process_device, ["hostname", "device_type", "notes"], system_prompt=system_prompt, device_prompt=device_prompt)

In [40]:
device_norm_df

,ip,hostname,fqdn,mac,owner,device_type,site,notes,ip_out,ip_valid,...,owner_email,owner_team,owner_issues,owner_recommended_action,owner_normalization_steps,device_out,device_type_confidence,device_issues,device_recommended_action,device_normalization_steps
source_row_id,,,,,,,,,,,,,,,,,,,,,
1,192.168.010.005,HOST01,NaN,AA-BB-CC-DD-EE-FF,priya (platform) priya@corp.example.com,server,BLR Campus,db host,192.168.10.5,True,...,priya@corp.example.com,platform,None,None,owner_trim|owner_parse,HOST01,high,None,None,device_trim|device_parse
2,10.0.1.300,host-02,host-02.local,11-22-33-44-55-66,ops,NaN,HQ Bldg 1,edge gw?,10.0.1.300,False,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,,,Missing device fields,Correct device or mark record for revision,device_trim|device_parse|device_invalid_missin...
3,10.0.1,host03,NaN,aabb.ccdd.eeff,jane@corp.example.com,switch,HQ-BUILDING-1,NaN,10.0.1,False,...,jane@corp.example.com,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,switch,low,None,None,device_trim|device_parse
4,10.0.1.1.2,printer-01,NaN,00:11:22:33:44:55,Facilities,printer,HQ,NaN,10.0.1.1.2,False,...,,Facilities,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,printer,high,None,None,device_trim|device_parse
5,fe80::1%eth0,iot-cam01,NaN,00:aa:bb:cc:dd:ee,sec,iot,Lab-1,camera PoE on port 3,fe80::1%eth0,False,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,iot-cam01,high,None,None,device_trim|device_parse
6,127.0.0.1,local-test,NaN,NaN,NaN,NaN,NaN,NaN,127.0.0.1,True,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,local-test,low,None,None,device_trim|device_parse
7,169.254.10.20,host-apipa,NaN,NaN,NaN,NaN,NaN,NaN,169.254.10.20,True,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,host-apipa,low,None,None,device_trim|device_parse
8,10.10.10.10,srv-10,NaN,NaN,platform,server,BLR campus,NaN,10.10.10.10,True,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,server,high,None,None,device_trim|device_parse
9,abc.def.ghi.jkl,badhost,NaN,NaN,NaN,NaN,NaN,NaN,abc.def.ghi.jkl,False,...,,,Missing owner fields,Correct owner or mark record for revision,owner_trim|owner_parse|owner_invalid_missing_o...,badhost,low,None,None,device_trim|device_parse


# Gather anomalies

In [41]:
def collect_anomalies(df: pd.DataFrame) -> List[Dict]:
    anomaly_records = []
    for _, row in df.iterrows():
        source_row_id = row.name

        issues = []
        for col in row.index:
            if col.endswith("issues"):
                issue_val = row[col]
                if pd.notna(issue_val) and str(issue_val).strip().lower() != "none":
                    issue_field = col[:col.index("_issues")]
                    issues.append(
                        {
                            "field": issue_field,
                            "type": issue_val,
                            "value": row[f"{issue_field}_out"] if pd.notna(row[f"{issue_field}_out"]) else ""
                        }
                    )
        recommended_actions = []
        for col in row.index:
            if col.endswith("recommended_action"):
                recommended_action = row[col]
                if pd.notna(recommended_action) and str(recommended_action).strip().lower() != "none":
                    recommended_actions.append(recommended_action)
        
        anomaly_records.append(
            {
                "source_row_id": source_row_id,
                "issues": issues,
                "recommended_actions": recommended_actions
            }
        )

    return anomaly_records

In [42]:
def generate_anomalies_json(output_file: str, anomaly_records: Dict) -> None:
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(anomaly_records, f, ensure_ascii=False, indent=2)

In [43]:
anomaly_records = collect_anomalies(device_norm_df)
generate_anomalies_json("anomalies.json", anomaly_records)

# Clean df according to schema

In [44]:
list(device_norm_df.columns)

['ip',
 'hostname',
 'fqdn',
 'mac',
 'owner',
 'device_type',
 'site',
 'notes',
 'ip_out',
 'ip_valid',
 'ip_version',
 'ip_reverse_ptr',
 'ip_classification',
 'subnet_cidr',
 'ip_issues',
 'ip_recommended_action',
 'ip_normalization_steps',
 'mac_out',
 'mac_valid',
 'mac_kind',
 'mac_issues',
 'mac_recommended_action',
 'mac_normalization_steps',
 'site_out',
 'site_issues',
 'site_recommended_action',
 'site_normalization_steps',
 'hostname_out',
 'hostname_valid',
 'hostname_kind',
 'hostname_issues',
 'hostname_recommended_action',
 'hostname_normalization_steps',
 'fqdn_out',
 'fqdn_valid',
 'fqdn_kind',
 'fqdn_issues',
 'fqdn_recommended_action',
 'fqdn_normalization_steps',
 'owner_out',
 'owner_email',
 'owner_team',
 'owner_issues',
 'owner_recommended_action',
 'owner_normalization_steps',
 'device_out',
 'device_type_confidence',
 'device_issues',
 'device_recommended_action',
 'device_normalization_steps']

In [45]:
normalization_steps_columns = [c for c in device_norm_df.columns if c.endswith("normalization_steps")]

In [46]:
device_norm_df["normalization_steps"] = device_norm_df[normalization_steps_columns].fillna("").agg("|".join, axis=1)

In [47]:
columns_of_interest = [
    'notes',
    'ip_out',
    'ip_valid',
    'ip_version',
    'ip_reverse_ptr',
    'ip_classification',
    'subnet_cidr',
    'mac_out',
    'mac_valid',
    'mac_kind',
    'site_out',
    'hostname_out',
    'hostname_valid',
    'hostname_kind',
    'fqdn_out',
    'fqdn_valid',
    'fqdn_kind',
    'owner_out',
    'owner_email',
    'owner_team',
    'device_out',
    'device_type_confidence',
]

In [48]:
clean_df = device_norm_df[columns_of_interest]

In [49]:
clean_df.rename(
    columns = {
        col: col[:col.index("_out")] for col in clean_df.columns if col.endswith("_out")
    }, inplace=True
)

/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_8510/569458070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.rename(


In [50]:
clean_df

,notes,ip,ip_valid,ip_version,ip_reverse_ptr,ip_classification,subnet_cidr,mac,mac_valid,mac_kind,...,hostname_valid,hostname_kind,fqdn,fqdn_valid,fqdn_kind,owner,owner_email,owner_team,device,device_type_confidence
source_row_id,,,,,,,,,,,,,,,,,,,,,
1,db host,192.168.10.5,True,4,5.10.168.192.in-addr.arpa,private,192.168.10.5/24,aa:bb:cc:dd:ee:ff,True,eui48,...,True,single_label,NaN,False,,Priya,priya@corp.example.com,platform,HOST01,high
2,edge gw?,10.0.1.300,False,,,,,11:22:33:44:55:66,True,eui48,...,True,single_label,host-02.local,True,fqdn,,,,,
3,NaN,10.0.1,False,,,,,aa:bb:cc:dd:ee:ff,True,eui48,...,True,single_label,NaN,False,,Jane,jane@corp.example.com,,switch,low
4,NaN,10.0.1.1.2,False,,,,,00:11:22:33:44:55,True,eui48,...,True,single_label,NaN,False,,,,Facilities,printer,high
5,camera PoE on port 3,fe80::1%eth0,False,,,,,00:aa:bb:cc:dd:ee,True,eui48,...,True,single_label,NaN,False,,,,,iot-cam01,high
6,NaN,127.0.0.1,True,4,1.0.0.127.in-addr.arpa,loopback,127.0.0.0/8,nan,False,,...,True,single_label,NaN,False,,,,,local-test,low
7,NaN,169.254.10.20,True,4,20.10.254.169.in-addr.arpa,link_local_apipa,169.254.0.0/16,nan,False,,...,True,single_label,NaN,False,,,,,host-apipa,low
8,NaN,10.10.10.10,True,4,10.10.10.10.in-addr.arpa,private,10.10.10.10/24,nan,False,,...,True,single_label,NaN,False,,,,,server,high
9,NaN,abc.def.ghi.jkl,False,,,,,nan,False,,...,True,single_label,NaN,False,,,,,badhost,low


In [51]:
clean_df.to_csv("inventory_clean.csv", index=True)